In [ ]:
# bank_churn_nn.py
# Minimal dependencies: pandas, numpy, scikit-learn, tensorflow
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

# 1) Read the dataset
df = pd.read_csv("Churn_Modelling.csv")  # make sure file is in same folder

# Quick peek (optional)
# print(df.head())

# 2) Distinguish feature and target set
# Drop columns that are identifiers or not useful: RowNumber, CustomerId, Surname
X = df.drop(columns=["RowNumber", "CustomerId", "Surname", "Exited"])
y = df["Exited"]

# Handle categorical variables:
# - Gender -> map to 0/1
# - Geography -> one-hot encoding (drop first to avoid dummy trap)
X["Gender"] = X["Gender"].map({"Male": 1, "Female": 0})
X = pd.get_dummies(X, columns=["Geography"], drop_first=True)

# 2b) Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X.values, y.values, test_size=0.2, random_state=42, stratify=y.values
)

# 3) Normalize train and test data (StandardScaler)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 4) Initialize and build the model (with improvements)
def make_model(input_dim):
    model = Sequential()
    model.add(Dense(64, input_dim=input_dim, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Dense(32, activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(16, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))  # binary output

    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy"]
    )
    return model

model = make_model(X_train.shape[1])

# Early stopping to avoid overfitting
es = EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)

# Fit model
history = model.fit(
    X_train, y_train,
    validation_split=0.15,
    epochs=100,
    batch_size=32,
    callbacks=[es],
    verbose=1
)

# 5) Evaluate: print accuracy and confusion matrix
y_pred_prob = model.predict(X_test).ravel()
y_pred = (y_pred_prob >= 0.5).astype(int)

acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

print("Test Accuracy:", acc)
print("Confusion Matrix:\n", cm)
